In [ ]:
import pandas as pd
import numpy as np
from scipy import stats 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.formula.api import ols



# Словарь файлов и соответствующих им названий столбцов
files = {
    'tksg.xlsx': 'TCSG',
    'yandex.xlsx': 'YDEX',
    'vk.xlsx': 'VKCO',
    'rostelecom.xlsx': 'RTKM',
    'usd_rub.xlsx': 'USD_RUB',
    'brent.xlsx': 'Brent',
    'bitcoin.xlsx': 'BTC_RUB'
}

# 1. Загрузка данных
dfs = []

for file, col_name in files.items():
    df = pd.read_excel(file)
    df.columns = ['Дата', col_name]
    df['Дата'] = pd.to_datetime(df['Дата'], dayfirst=True).dt.date  # Только дата, без времени
    df.set_index('Дата', inplace=True)
    dfs.append(df)

# 2. Объединение всех DataFrame по индексу (дате) через inner join
combined_df = pd.concat(dfs, axis=1, join='inner')
combined_df.reset_index(inplace=True)
combined_df.rename(columns={'index': 'Дата'}, inplace=True)

# Сохраняем объединённые данные в Excel
combined_df.to_excel('combined_data.xlsx', index=False)

# 3. Преобразуем числовые столбцы к float (корректная обработка запятых)
numeric_cols = combined_df.columns[1:]

for col in numeric_cols:
    combined_df[col] = pd.to_numeric(
        combined_df[col].astype(str).str.replace(',', '.', regex=False),
        errors='coerce'
    )

# 4. Удаление выбросов методом IQR
cleaned_df = combined_df.copy()

for col in numeric_cols:
    Q1 = cleaned_df[col].quantile(0.25)
    Q3 = cleaned_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Замена выбросов на границы
    cleaned_df[col] = np.where(cleaned_df[col] < lower_bound, lower_bound, cleaned_df[col])
    cleaned_df[col] = np.where(cleaned_df[col] > upper_bound, upper_bound, cleaned_df[col])

# Сохраняем очищенные данные в Excel
cleaned_df.to_excel('cleaned_data.xlsx', index=False)

# 5. Нормализация Z-Score
normalized_df = cleaned_df.copy()

for col in numeric_cols:
    mean = normalized_df[col].mean()
    std = normalized_df[col].std()
    normalized_df[col] = (normalized_df[col] - mean) / std

# Сохраняем нормализованные данные в Excel
normalized_df.to_excel('normalized_data.xlsx', index=False)

# --- Вывод первых и последних 5 строк для каждого датасета ---
print("🔹 combined_df — первые 5 строк:")
print(combined_df.head().to_string(index=False))
print("\n🔹 combined_df — последние 5 строк:")
print(combined_df.tail().to_string(index=False))

print("\n\n🔹 cleaned_df — первые 5 строк:")
print(cleaned_df.head().to_string(index=False))
print("\n🔹 cleaned_df — последние 5 строки:")
print(cleaned_df.tail().to_string(index=False))

print("\n\n🔹 normalized_df — первые 5 строк:")
print(normalized_df.head().to_string(index=False))
print("\n🔹 normalized_df — последние 5 строки:")
print(normalized_df.tail().to_string(index=False))

In [ ]:

factors = ['TCSG', 'YDEX', 'VKCO', 'RTKM', 'USD_RUB', 'Brent', 'BTC_RUB']

for factor in factors:
    plt.figure(figsize=(14, 6))

    # Гистограмма (слева)
    plt.subplot(1, 2, 1)
    sns.histplot(normalized_df[factor], kde=True, bins=30)  # Исправлено: normalized_df[factor]
    plt.title(f'Гистограмма {factor}')
    plt.xlabel('Z-нормализованное значение')

    # Q-Q plot (справа)
    plt.subplot(1, 2, 2)
    sm.qqplot(normalized_df[factor].dropna(), line='s', ax=plt.gca())  # Исправлено: normalized_df[factor]
    plt.title(f'Q-Q plot {factor}')

    plt.tight_layout()
    plt.show()

In [ ]:

# Загрузка нормализованного датафрейма или его использование напрямую после предыдущих шагов
# normalized_df — должен быть доступен как DataFrame с числовыми столбцами

factors = ['TCSG', 'YDEX', 'VKCO', 'RTKM', 'USD_RUB', 'Brent', 'BTC_RUB']

# Создаем матрицу корреляций
corr_matrix = pd.DataFrame(index=factors, columns=factors, dtype=float)
p_matrix = pd.DataFrame(index=factors, columns=factors, dtype=float)

for i in factors:
    for j in factors:
        corr, p_value = spearmanr(normalized_df[i], normalized_df[j])
        corr_matrix.loc[i, j] = round(corr, 2)
        p_matrix.loc[i, j] = round(p_value, 4)

print("\n🔹 Матрица ранговой корреляции Спирмена:")
print(corr_matrix)

print("\n🔹 Матрица p-value для корреляций:")
print(p_matrix)

# Сохранение результатов в Excel (опционально)
corr_matrix.to_excel('spearman_correlation_matrix.xlsx')
p_matrix.to_excel('spearman_pvalue_matrix.xlsx')

In [ ]:


# Предполагается, что normalized_df уже создан в предыдущих шагах
# Если нужно загрузить из файла, раскомментируйте:
# normalized_df = pd.read_excel('normalized_data.xlsx')

# Список зависимых и независимых переменных
dependent_vars = ['TCSG', 'YDEX', 'VKCO', 'RTKM']
independent_vars = ['USD_RUB', 'Brent', 'BTC_RUB']
X = normalized_df[independent_vars]

# Словарь для хранения результатов
results = {}

# Построение моделей и визуализаций
for y_var in dependent_vars:
    y = normalized_df[y_var]
    
    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Обучение модели
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Прогнозирование
    y_pred = model.predict(X_test)
    
    # Оценка качества
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    # Сохранение результатов
    results[y_var] = {
        'coefficients': dict(zip(independent_vars, model.coef_)),
        'intercept': model.intercept_,
        'MAE': mae,
        'MSE': mse,
        'y_test': y_test,
        'y_pred': y_pred,
        'X_test': X_test
    }
    
    # График 1: Реальные vs Предсказанные значения
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.values, label='Реальные значения', color='blue', marker='o', linestyle='-', alpha=0.6)
    plt.plot(y_pred, label='Предсказанные значения', color='red', marker='x', linestyle='--', alpha=0.6)
    plt.title(f'Реальные и предсказанные значения для {y_var}')
    plt.xlabel('Номер наблюдения в тестовой выборке')
    plt.ylabel('Z-нормализованная цена акций')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'prediction_plot_{y_var}.png')
    plt.show()
    
    # График 2: Гистограмма ошибок
    errors = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.histplot(errors, kde=True, bins=30, color='purple')
    plt.title(f'Распределение ошибок для {y_var}')
    plt.xlabel('Ошибка (реальное - предсказанное)')
    plt.ylabel('Частота')
    plt.grid(True)
    plt.savefig(f'error_histogram_{y_var}.png')
    plt.show()

# Графики регрессии (диаграммы рассеяния с линиями регрессии)
for y_var in dependent_vars:
    y = normalized_df[y_var]
    for x_var in independent_vars:
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=normalized_df[x_var], y=y, color='blue', alpha=0.5)
        # Линия регрессии
        z = np.polyfit(normalized_df[x_var], y, 1)
        p = np.poly1d(z)
        plt.plot(normalized_df[x_var], p(normalized_df[x_var]), color='red', linestyle='--')
        plt.title(f'Зависимость {y_var} от {x_var}')
        plt.xlabel(f'{x_var} (Z-нормализованное)')
        plt.ylabel(f'{y_var} (Z-нормализованное)')
        plt.grid(True)
        plt.savefig(f'regression_plot_{y_var}_{x_var}.png')
        plt.show()

# Вывод результатов
for y_var, metrics in results.items():
    print(f"\nМодель для {y_var}:")
    print("Коэффициенты регрессии:")
    for feature, coef in metrics['coefficients'].items():
        print(f"  {feature}: {coef:.4f}")
    print(f"Свободный член (intercept): {metrics['intercept']:.4f}")
    print(f"Средняя абсолютная ошибка (MAE): {metrics['MAE']:.4f}")
    print(f"Средняя квадратичная ошибка (MSE): {metrics['MSE']:.4f}")

# Сохранение результатов в Excel
results_df = pd.DataFrame({
    'Компания': [],
    'USD_RUB (β₁)': [],
    'Brent (β₂)': [],
    'BTC_RUB (β₃)': [],
    'Intercept': [],
    'MAE': [],
    'MSE': []
})

for y_var, metrics in results.items():
    row = {
        'Компания': y_var,
        'USD_RUB (β₁)': metrics['coefficients']['USD_RUB'],
        'Brent (β₂)': metrics['coefficients']['Brent'],
        'BTC_RUB (β₃)': metrics['coefficients']['BTC_RUB'],
        'Intercept': metrics['intercept'],
        'MAE': metrics['MAE'],
        'MSE': metrics['MSE']
    }
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

results_df.to_excel('regression_results.xlsx', index=False)
print("\nРезультаты сохранены в 'regression_results.xlsx'")

In [ ]:


# Подготовка данных
dependent_vars = ['TCSG', 'YDEX', 'VKCO', 'RTKM']
results_anova = []
columns = ['Source', 'sum_sq', 'df', 'F', 'PR(>F)']

for y_var in dependent_vars:
    # Формирование модели
    formula = f'{y_var} ~ USD_RUB + Brent + BTC_RUB'
    model = ols(formula, data=normalized_df).fit()
    
    # Дисперсионный анализ
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    # Форматирование таблицы
    anova_table = anova_table.reset_index().rename(columns={'index': 'Source'})
    anova_table['sum_sq'] = anova_table['sum_sq'].round(4)
    anova_table['F'] = anova_table['F'].round(4)
    anova_table['PR(>F)'] = anova_table['PR(>F)'].round(6)
    
    # Добавление заголовка
    header_row = pd.DataFrame([[f'Результаты ANOVA для {y_var}', '', '', '', '']], columns=columns)
    results_anova.append(header_row)
    results_anova.append(anova_table)
    
    # Вывод результатов
    print(f"\nРезультаты ANOVA для {y_var}:")
    print(anova_table)

# Объединение всех результатов
final_df = pd.concat(results_anova, ignore_index=True)

# Сохранение в Excel
final_df.to_excel('anova_results.xlsx', sheet_name='ANOVA_Results', index=False)